In [137]:
import sys
sys.path.append('../_amigocloud')

from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json

# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
#verifica si esta el archivo de autenticacion
#si no, pide autenticacion y guarda las credenciales
gauth.LoadCredentialsFile("mycreds.txt")
if gauth.credentials is None:
    gauth.LocalWebserverAuth()
elif gauth.access_token_expired:
    gauth.Refresh()
else:
    gauth.Authorize()
gauth.SaveCredentialsFile("mycreds.txt")
gauth.LocalWebserverAuth()  # Abre una ventana del navegador para autenticarse con tu cuenta de Google
drive = GoogleDrive(gauth)

In [138]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [139]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [140]:
def cargar_tokens(t_name, ruta_tkn):
    with open(ruta_tkn, 'r') as f:
        tokens = json.load(f)
    return tokens.get(t_name)

In [141]:
# RUTAS
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

# tkns
tkns = cargar_tokens('tkns_amigocloud', os.path.join(ruta_completa + r'\_keys\tkns.json'))
tkn = tkns['cargar_informes']

amigocloud = AmigoCloud(token=tkn)
amigocloud

In [142]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            try:
                convert(ruta_docx, ruta_pdf)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

In [143]:
data_siglas = {
    'EMPC': {
        'tipo' : 'Evaluacion de mantenimiento preventivo',
        'id_propject' : 34080,
        'dataset' : 'dataset_341171',
        'drive' : '1GFQMfnFYdy9iDj8h8Mhqh_5ZMKMJeKka', 
        'campo' : 'link_reporte'
    },
    'CDSE': {
        'tipo' : 'Calidad de semilla',
        'id_propject' : 35567,
        'dataset' : 'dataset_365887',
        'drive' : '_CALIDAD DE SEMILLA', 
        'campo' : 'link_informe'
    }
}

In [144]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        file = drive.CreateFile({
            "mimeType": "application/pdf",
            "parents": [{"kind": "drive#fileLink", "id": id_drive}]
        })
        file['title'] = title_file
        file.SetContentFile(path_file)
        file.Upload()
        print('Created file %s with mimeType %s' % (file['title'], file['mimeType']))
        file.InsertPermission({
            'type': 'anyone',
            'value': 'anyone',
            'role': 'reader'
        })
        return file['alternateLink']
    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [149]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [150]:
contenido = os.listdir(directorio_destino)
contenido

['633_EMPC_01-04-2025_AGROP. VICENTE ROCA GIL SRL_105.pdf',
 '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_100.pdf',
 '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_101.pdf',
 '9677_EMPC_02-04-2025_OVANDO JIMENEZ JAIRO ESTEVAN_109.pdf']

In [151]:
def cargar_link_to_amigocloud(dataset, campo, url, id_proyecto, id):
    query = {'query': f"UPDATE {dataset} SET {campo} = '{url}' WHERE id = {id}"}
    try:
        res = amigocloud.post(
            f'https://app.amigocloud.com/api/v1/projects/{id_proyecto}/sql',
            query
        )
        return True
    except Exception as e:
        print(f"[EXCEPCIÓN] Error al ejecutar la consulta: {e}")
        return False

In [152]:
for file in contenido:
    partes = file.split('_')
    
    id = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file

    info_sigla = data_siglas[sigla]
    id_drive = info_sigla['drive']
    dataset = info_sigla['dataset']
    campo = info_sigla['campo']
    id_proyecto = info_sigla['id_propject']
    
    path_file = os.path.join(directorio_destino, file)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    result_amigocloud = cargar_link_to_amigocloud(dataset, campo, url_drive, id_proyecto, id)
    if result_amigocloud == False:
        continue
    print(url_drive)

[INFO] El archivo '633_EMPC_01-04-2025_AGROP. VICENTE ROCA GIL SRL_105.pdf' ya existe. Se reemplazará.
[OK] Archivo '633_EMPC_01-04-2025_AGROP. VICENTE ROCA GIL SRL_105.pdf' subido exitosamente.
https://drive.google.com/file/d/1VYPAw2sorK1ayGXxMkCBb6sqvxceFojT/view?usp=drivesdk
[INFO] El archivo '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_100.pdf' ya existe. Se reemplazará.
[OK] Archivo '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_100.pdf' subido exitosamente.
https://drive.google.com/file/d/17uSGz-wJiYS59KjO6bSY8XBwBW_afKjg/view?usp=drivesdk
[INFO] El archivo '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_101.pdf' ya existe. Se reemplazará.
[OK] Archivo '75_EMPC_31-03-2025_AGUILERA TARADELLES NELSON MARIANO_101.pdf' subido exitosamente.
https://drive.google.com/file/d/1zDYydC0_1ehWXUEzDh5IqN019VHMW5hD/view?usp=drivesdk
[INFO] El archivo '9677_EMPC_02-04-2025_OVANDO JIMENEZ JAIRO ESTEVAN_109.pdf' ya existe. Se reemplazará.
[OK] Archivo '9677_EMPC_02-04-2025